## 隐马尔可夫模型

### 盒子摸球模型的HMM观测序列生成

In [1]:
import numpy as np

# HMM类
class HMM:
    def __init__(self, N, M, pi=None, A=None, B=None):
        self.N = N # 可能的状态数
        self.M = M # 可能的观测数
        self.pi = pi # 初始状态概率向量
        self.A = A # 状态转移矩阵
        self.B = B # 观测概率矩阵
    
    def rdistribution(self, dist):
        '''根据给定的概率分布随机返回数据'''
        r = np.random.rand()
        for ix, p in enumerate(dist):
            if r < p:
                return ix
            r -= p
    
    def generate(self, T):
        '''生成HMM观测序列'''
        i = self.rdistribution(self.pi) # 根据初始概率分布生成第一个状态
        o = self.rdistribution(self.B[i]) # 生成第一个观测数据
        observed_data = [o]

        for _ in range(T-1): # 遍历生成后续的状态和观测数据
            i = self.rdistribution(self.A[i])
            o = self.rdistribution(self.B[i])
            observed_data.append(o)
        return observed_data
    
pi = np.array([0.25, 0.25, 0.25, 0.25]) # 初始状态概率矩阵

# 状态转移矩阵
A = np.array([
    [0, 1, 0, 0],
    [0.4, 0, 0.6, 0],
    [0, 0.4, 0, 0.6],
    [0, 0, 0.5, 0.5]
])

# 观测概率矩阵
B = np.array([
    [0.5, 0.5],
    [0.6, 0.4],
    [0.2, 0.8],
    [0.3, 0.7]
])

N = 4 # 状态数，
M = 2 # 观测数，

hmm = HMM(N, M, pi, A, B) # 创建HMM实例
hmm.generate(5)

[0, 0, 1, 1, 0]

### 基于盒子摸球实验的前向算法实现

In [30]:
# 前向算法计算条件概率
def prob_calc(O): 
    '''
    输入：
    O：观测序列
    输出：
    alpha.sum()：条件概率
    '''

    alpha = pi * B[:, O[0]] # 初始值

    for o in O[1:]:
        alpha_next = np.empty(4)
        for j in range(4):
            alpha_next[j] = np.sum(A[:, j] * alpha * B[j, o])
        alpha = alpha_next
    
    return alpha.sum()

O = [1, 0, 1, 0, 0] # 给定观测
print(prob_calc(O)) # 计算生成该观测的概率

0.01983169125


### 基于盒子摸球实验的后向算法实现

In [33]:
# 后向算法计算条件概率
def prob_calc(O): 
    '''
    输入：
    O：观测序列
    输出：
    beta.sum()：条件概率
    '''

    beta = pi * B[:, O[-1]] # 初始值

    for o in O[3::-1]:
        beta_prev = np.empty(4)
        for j in range(3,-1,-1):
            beta_prev[j] = np.sum(A[:, j] * beta * B[j, o])
        beta = beta_prev
    
    return beta.sum()

O = [1, 0, 1, 0, 0] # 给定观测
print(prob_calc(O)) # 计算生成该观测的概率

0.027407493749999998


### 基于盒子摸球实验的维特比算法实现

In [41]:
# 序列标注问题和维特比算法
def verterbi_decode(O):
    '''
    输入：
    O：观测序列
    输出：
    path：最优隐状态路径
    '''

    T, o = len(O), O[0] # 序列长度和初始观测
    delta = pi * B[:, o] # 初始化delta变量
    varphi = np.zeros((T, 4), dtype=int) # 初始化varphi变量
    path = [0] * T

    # 递推
    for i in range(1, T):
        delta = delta.reshape(-1, 1)
        tmp = delta * A
        varphi[i, :] = np.argmax(tmp, axis=0)
        delta = np.max(tmp, axis=0) * B[:, O[i]]
    
    path[-1] = np.argmax(delta)
    
    # 回溯最优路径
    for i in range(T-1, 0, -1):
        path[i-1] = varphi[i, path[i]]
    return path

O = [1, 0, 1, 1, 0] # 给定观测序列
print(verterbi_decode(O)) # 输出最有可能的隐状态序列

[0, 1, 2, 3, 3]
